##드라이브 연결

In [146]:
import pandas as pd

In [147]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "gdrive/MyDrive/ML/Recommendation/Data/Movielens/"
os.listdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


['movies.csv',
 'ratings.csv',
 'tags.csv',
 'links.csv',
 'movies_refined.csv',
 'ratings_refined.csv',
 'user_based_cf_prediction.csv',
 '1207 sample_data_pp.csv',
 '1208 CosSim_Genres.csv',
 '1208 CosSim_All.csv',
 '1208 CosSim_Story.csv',
 '1208 CosSim_Title.csv',
 '1208 CosSim_Tag.csv',
 '1208 EucSim_Title.csv',
 '1208 EucSim_Tag.csv',
 '1208 EucSim_Story.csv',
 '1208 EucSim_Genres.csv',
 '1208 EucSim_All.csv']

In [148]:
cosSim_All = pd.read_csv(path + '1208 CosSim_All.csv')
cosSim_Genres =pd.read_csv(path + '1208 CosSim_Genres.csv')
cosSim_Title =pd.read_csv(path + '1208 CosSim_Title.csv')
cosSim_Tag =pd.read_csv(path + '1208 CosSim_Tag.csv')
cosSim_Story =pd.read_csv(path + '1208 CosSim_Story.csv')

In [149]:
EucSim_All =  pd.read_csv(path + '1208 EucSim_All.csv')
EucSim_Genres = pd.read_csv(path + '1208 EucSim_Genres.csv')
EucSim_Title = pd.read_csv(path + '1208 EucSim_Title.csv')
EucSim_Tag = pd.read_csv(path + '1208 EucSim_Tag.csv')
EucSim_Story = pd.read_csv(path + '1208 EucSim_Story.csv')

In [150]:
k = pd.read_csv(path + "1207 sample_data_pp.csv")
movie = pd.read_csv(path + "movies_refined.csv")
rating = pd.read_csv(path + 'ratings.csv')

##파일 병합 및 전 처리

In [151]:
new_data = pd.merge(movie, k, on='movieId', how='right')

In [152]:
new_df = pd.merge(movie, k, on='movieId', how='left')

In [153]:
new_ratings = pd.merge(rating, k, on='movieId', how='left')

In [154]:
title_list = new_data['title_x'].tolist()

In [155]:
movie_id_list = new_data['movieId'].tolist()

###유클리디안 행렬 인덱스 설정

In [156]:
EucSim_All_id   =EucSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_All.columns, movie_id_list)))
EucSim_Genres_id=EucSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Genres.columns, movie_id_list)))
EucSim_Title_id =EucSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Title.columns, movie_id_list)))
EucSim_Tag_id   =EucSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Tag.columns, movie_id_list)))
EucSim_Story_id =EucSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Story.columns, movie_id_list)))

In [157]:
EucSim_All   = EucSim_All.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_All.columns, title_list)))
EucSim_Genres= EucSim_Genres.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Genres.columns, title_list)))
EucSim_Title = EucSim_Title.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Title.columns, title_list)))
EucSim_Tag   = EucSim_Tag.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Tag.columns, title_list)))
EucSim_Story = EucSim_Story.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Story.columns, title_list)))

###코사인 행렬 인덱스 설정

In [158]:
cosSim_All_id   =cosSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_All.columns, movie_id_list)))
cosSim_Genres_id=cosSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Genres.columns, movie_id_list)))
cosSim_Title_id =cosSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Title.columns, movie_id_list)))
cosSim_Tag_id   =cosSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Tag.columns, movie_id_list)))
cosSim_Story_id =cosSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Story.columns, movie_id_list)))

In [159]:
cosSim_All_id   =cosSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_All.columns, movie_id_list)))
cosSim_Genres_id=cosSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Genres.columns, movie_id_list)))
cosSim_Title_id =cosSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Title.columns, movie_id_list)))
cosSim_Tag_id   =cosSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Tag.columns, movie_id_list)))
cosSim_Story_id =cosSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Story.columns, movie_id_list)))

###레이팅 전처리

In [160]:
new_ratings_refined = rating[rating['movieId'].isin(movie_id_list)]

In [161]:
new_ratings_refined = new_ratings_refined[['userId', 'movieId', 'rating']]

##모델링

In [162]:
from tqdm.notebook import tqdm
import numpy as np
def modeling(similarity_matrix, data):
    df_pred_all = pd.DataFrame()
    titles = title_list
    all_users = sorted(data['userId'].unique())
    n_titles = len(titles)

    for user in tqdm(all_users):
        idx = data[data['userId'] == user].index

        # 유사도
        watched_movies = data.loc[idx, 'movieId'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_movies]
        sub_sim_mat = sub_sim_mat.T.to_numpy()
        sim_N = np.sum(sub_sim_mat, axis=1) + 1


        # 평점 예측
        watched_movies_y = data.loc[idx, 'rating']
        watched_movies_y = np.array(watched_movies_y.tolist()).reshape(-1, 1)

        pred_y = np.matmul(sub_sim_mat, watched_movies_y).flatten() / sim_N

        users_list = [user] * n_titles
        cur_pred = pd.DataFrame(zip(users_list, titles, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)
    return df_pred_all

##평점 예측

###코사인 유사도 기반

####all

In [163]:
cosSim_All_id

,1,2,3,4,5,6,7,8,9,10,...,3324,3325,3326,3327,3328,3329,3330,3331,3334,3338
1,1.000000,0.785750,0.650031,0.466738,0.814383,0.516607,0.344824,0.797125,0.457293,0.322287,...,0.658534,0.375459,0.536232,0.674771,0.343047,0.398148,0.378806,0.491020,0.436336,0.596340
2,0.787437,1.000000,0.573271,0.584012,0.678320,0.536172,0.471093,0.907093,0.615181,0.506260,...,0.502899,0.400609,0.576293,0.773003,0.344911,0.494552,0.476329,0.619066,0.545249,0.497770
3,0.662085,0.584697,1.000000,0.540845,0.707638,0.491259,0.520871,0.566345,0.450563,0.417607,...,0.671176,0.470040,0.519406,0.644680,0.364602,0.427901,0.416773,0.467399,0.589594,0.537724
4,0.375624,0.483880,0.426860,1.000000,0.449428,0.714500,0.720298,0.505553,0.830274,0.641777,...,0.355610,0.729016,0.453677,0.581487,0.707574,0.425422,0.841649,0.488473,0.327588,0.299039
5,0.811703,0.671085,0.692837,0.535496,1.000000,0.450501,0.361283,0.742262,0.453223,0.281668,...,0.746989,0.454780,0.605773,0.712820,0.393027,0.440683,0.426178,0.540661,0.463441,0.610274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3329,0.350219,0.449968,0.360308,0.470957,0.404736,0.209354,0.341255,0.521785,0.348985,0.294301,...,0.435488,0.423753,0.532692,0.605304,0.376089,1.000000,0.557459,0.806546,0.468986,0.580812
3330,0.236992,0.333691,0.250880,0.827166,0.296669,0.562795,0.550913,0.403572,0.691501,0.504742,...,0.310480,0.720575,0.432158,0.480618,0.616005,0.466254,1.000000,0.531742,0.469920,0.328877
3331,0.459067,0.563123,0.419234,0.551629,0.507848,0.324801,0.290043,0.627823,0.456842,0.296992,...,0.555462,0.531025,0.639620,0.721214,0.394097,0.744318,0.623948,1.000000,0.616581,0.696556
3334,0.395931,0.486651,0.512486,0.389149,0.427777,0.352176,0.327433,0.482711,0.375292,0.364689,...,0.473721,0.486808,0.500513,0.573645,0.279730,0.466801,0.558801,0.602622,1.000000,0.484922


In [164]:
df_pred_cos_all= modeling(cosSim_All_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####스토리

In [165]:
df_pred_cos_story=modeling(cosSim_Story_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####장르

In [166]:
df_pred_cos_genre=modeling(cosSim_Genres_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####타이틀

In [167]:
df_pred_cos_title=modeling(cosSim_Title_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####태그

In [168]:
df_pred_cos_tag=modeling(cosSim_Tag_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####스토리 * 장르

####스토리 * 타이틀

####스토리 * 태그

####장르 * 타이틀

####장르 * 태그

####타이틀 * 태그

####스토리 * 타이틀 * 장르 * 태그

###유클리디안 기반

####All

In [169]:
df_pred_euc_all=modeling(EucSim_All_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####스토리

In [170]:
df_pred_euc_story=modeling(EucSim_Story_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####장르

In [171]:
df_pred_euc_genere=modeling(EucSim_Genres_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####타이틀

In [172]:
df_pred_euc_title=modeling(EucSim_Title_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

####태그

In [173]:
df_pred_euc_tag=modeling(EucSim_Tag_id,new_ratings_refined)

  0%|          | 0/609 [00:00<?, ?it/s]

##추천

###코사인 기반

####All

In [174]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_all(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_all[df_pred_cos_all['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####스토리

In [175]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_story(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_story[df_pred_cos_story['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####장르

In [176]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_genre(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_genre[df_pred_cos_genre['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####타이틀

In [177]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_title(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_title[df_pred_cos_title['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####태그

In [178]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_tag(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_tag[df_pred_cos_tag['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

###유클리디안 기반

####All

In [179]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_all(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_all[df_pred_euc_all['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####스토리

In [180]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_story(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_story[df_pred_euc_story['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####장르

In [181]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_genre(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_genre[df_pred_euc_genre['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####타이틀

In [182]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_title(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_title[df_pred_euc_title['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

####태그

In [183]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_tag(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_tag[df_pred_euc_tag['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()